### Face Mask Detector 
<img src="https://user-images.githubusercontent.com/74012107/130773556-28eb52f0-e368-4506-82eb-75a564754664.png" width="1000" height="700">

In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import cv2
import numpy as np

In [2]:
model = load_model('model.h5')

labels_dict = {0 : 'MASK', 1 : 'NO MASK'}
color_dict = {0 :(0, 255, 0), 1 : (0, 0, 255)} # green for 'MASK', red for 'NO MASK'

In [3]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [4]:
saved_images = 0
source = cv2.VideoCapture(0)

while(True):

    # Preprocess the image
    ret, image = source.read()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, 1.2, 5)  

    for (x, y, w, h) in faces:  
        face_image = gray[y : y + w, x : x + w]
        resized = cv2.resize(face_image, (100, 100))
        normalized = resized / 255.0
        reshaped = np.reshape(normalized, (1, 100, 100, 1))
        
        result = model.predict(reshaped)
        
        probability = round(result[0][np.argmax(result[0])], 3)
        label = np.argmax(result, axis = 1)[0]
      
        # Frame the face and print the result
        cv2.rectangle(image, (x,y), (x + w, y + h), color_dict[label], 2)
        cv2.rectangle(image, (x, y - 40), (x + w, y), color_dict[label], -1)
        cv2.putText(image, labels_dict[label] + ' ' + str(probability), 
                    (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        
    cv2.imshow('LIVE', image)
    key = cv2.waitKey(1)
    
    if(key == 27): # wait for 'ESC' key to exit
        break
    elif key == ord('s'): # wait for 's' key to save the image
        cv2.imwrite('FaceMaskDetector' + str(saved_images) + '.png', img)
        saved_images += 1
        
        
cv2.destroyAllWindows()
source.release()